In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/kaggle/input/mnist-digit-recognizer/train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# convert df into numpy array
df = np.array(df)
m, n = df.shape
print((m, n)) # m -> no of rows, n -> no of columns

(42000, 785)


In [4]:
df

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [5]:
# split data into train and cross-validation sets
np.random.shuffle(df)

# training set
df_train = df[0:33600].T
X_train = df_train[1:n]
Y_train = df_train[0]

df_cv = df[33600:42000].T
X_cv = df_cv[1:n]
Y_cv = df_cv[0]

print(f"training set : X_train:{X_train.shape}, Y_train:{Y_train.shape}, CV set: X_cv:{X_cv.shape}, Y_cv{Y_cv.shape}")

training set : X_train:(784, 33600), Y_train:(33600,), CV set: X_cv:(784, 8400), Y_cv(8400,)


In [6]:
# assign values for W_[1], b_[1] and W_[2], b_[2]
def init_params():
    W1 = np.random.randn(10, 784)
    b1 = np.random.randn(10, 1)
    W2 = np.random.randn(10, 10)
    b2 = np.random.randn(10, 1)
    return W1, b1, W2, b2

# define relu activation function
def ReLU(Z):
    return np.maximum(0, Z)

# define softmax function
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # stability
    return expZ / np.sum(expZ, axis=0, keepdims=True)

# define the forward propagation
def forward_prop(W1, b1, W2, b2, X):
    # for input layer
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
     
    # second hidden layer
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2

# one hot encoding
def one_hot_encoding(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

# ReLU function: if Z <= 0 -> slop is 0 : Z > 0 -> slop is 1
def deriv_ReLU(Z):
    return Z > 0

# define backward propagation
def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot_encoding(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

# update the new params based on each derivative
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [7]:
def loss(X, Y, A2):
    m = Y.size
    log_likelihood = np.log(A2[Y, range(m)])
    loss = 1 / m * np.sum(log_likelihood)
    return loss

def get_preds(A2):
    return np.argmax(A2, 0)

def get_accuracy(preds, Y):
    print(preds, Y)
    return np.sum(preds == Y) / Y.size

# Implement the Gradient Descent Algorithm for the Neural Network
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0: # print accuracy and iteration no per 10 rounds
            print(f"Iteration: {i}\nAccuracy: {get_accuracy(get_preds(A2), Y):.6f}")
    return W1, b1, W2, b2

In [8]:
# run gradient descent algorithm
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 100, 0.1)

[2 7 2 ... 7 2 2] [3 7 0 ... 7 7 4]
Iteration: 0
Accuracy: 0.128690
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 10
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 20
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 30
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 40
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 50
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 60
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 70
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 80
Accuracy: 0.097738
[4 4 4 ... 4 4 4] [3 7 0 ... 7 7 4]
Iteration: 90
Accuracy: 0.097738
